# Entrega 2 - Clasificador (Árboles de Decisión)

### Grupo 07:
     - Renzo Gambone C.I. 5.155.486-4
     - Germán Ouviña C.I. 4.823.566-1
     - Leandro Rodríguez C.I 4.691.736-4


## 1. Introducción
***
### 1.1. Objetivo
***
El objetivo de esta tarea fue A

En términos formales, los parámetros del problema se reducen a lo siguiente:
- **Tarea *T*:** Jugar una partida de Damas Chinas
- **Experiencia *E*:** Jugar contra un jugador aleatorio / Jugar contra sí mismo
- **Performance *P*:** Proporción de partidas ganadas

### 1.2. Entrega
***
La entrega de esta tarea consta de dos grandes componentes:
- **Informe** en formato de Jupyter Notebook (este informe).
- **Programa** simulador de entrenamiento y clasificación (y código fuente comentado).

El objetivo del informe es A

Por otra parte, el programa ofrece una interfaz en consola que permite A

*Acá hablamos de que conviene hacer pa entender la tarea*

### 1.3. Formato
***
En las siguientes secciones A

## 2. Diseño
***
En esta sección se detallan las características del diseño utilizado para construir el modelo A

### 2.1. Preprocesamiento
***
A

#### 2.1.1. Adaptación de función objetivo
***
A

#### 2.1.2. Interpretación de valores continuos
***
A

#### 2.1.3. Adaptación a bosque
***
A

### 2.2. Algoritmo
***
A

#### 2.2.1. Modelos
***
Acá mencionamos que usamos un arbol de decision y un bosque de arboles de decision

#### 2.2.2. Algoritmo de entrenamiento
***
Aca hablamos de DECISION como algoritmo de entrenamiento, mencionamos las diferencias en el entrenamiento del arbol y del bosque

#### 2.2.2.1. Árbol - Entrenamiento multiclase
***
A

#### 2.2.2.2. Bosque - Entrenamiento binario
***
A

#### 2.2.3. Algoritmo de clasificación
***
Aca hablamos de la clasificación en arboles de decision, mencionamos las diferencias en la clasificacion con un arbol único y con un bosque

#### 2.2.3.1. Árbol - Clasificación
***
A

#### 2.2.3.2. Bosque - Votación
***
A


### 2.3. Evaluación
***
A

#### 2.3.1. Métricas de evaluación
***
A

#### 2.3.2. Conjuntos de evaluación
***
A

## 3. Experimentación
***
A

### 3.1. Metodología
***

A

### 3.2. Recopilación y ajuste
***

A

### 3.3. Configuraciones paramétricas
***

#### 3.3.1. Instancias
***
A

#### 3.3.2. Parámetros
***
A

#### 3.3.3. Resultados
***
A

#### 3.3.3.1. Instancia 1
***
#### 3.3.3.1.1. Resultados de distintas configuraciones paramétricas
***

A
<table>
    <tr>
        <th>Iteraciones</th>
        <th>Turnos</th>
        <th>Ratio $\gamma$</th>
        <th>Pesos iniciales</th>
        <th>Pesos finales (aprox)</th>
        <th>Partidas ganadas</th>
        <th>Partidas perdidas</th>
        <th>Partidas empatadas</th>
        <th>Ratio de victorias</th>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.5</td>
        <td>[0.9, 0.9, 0.9, 0.9, 0.9]</td>
        <td>[0.50, 0.00, 1.00, 0.20, 0.42]</td>
        <td>13%</td>
        <td>7%</td>
        <td>80%</td>
        <td>0,65</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.5</td>
        <td>[0.1, 0.9, 0.1, 0.1, 0.1]</td>
        <td>[0.27, 0.00, 0.08, 0.91, 1.00]</td>
        <td>7%</td>
        <td>2%</td>
        <td>91%</td>
        <td>0,78</td>
    </tr>
    <tr style="background-color: #95dcd4;">
        <td>100</td>
        <td>100</td>
        <td>0.5</td>
        <td>[0.5, 0.5, 0.5, 0.5, 0.5]</td>
        <td>[0.68, 0.24, 0.00, 1.00, 0.38]</td>
        <td>21%</td>
        <td>6%</td>
        <td>73%</td>
        <td>0,78</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.9</td>
        <td>[0.9, 0.9, 0.9, 0.9, 0.9]</td>
        <td>[0.00, 1.00, 0.76, 0.69, 0.60]</td>
        <td>35%</td>
        <td>14%</td>
        <td>51%</td>
        <td>0,71</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.9</td>
        <td>[0.1, 0.9, 0.1, 0.1, 0.1]</td>
        <td>[0.83, 1.00, 0.65, 0.64, 0.00]</td>
        <td>58%</td>
        <td>24%</td>
        <td>18%</td>
        <td>0,70</td>
    </tr>
    <tr style="background-color: #95dcd4;">
        <td>100</td>
        <td>100</td>
        <td>0.9</td>
        <td>[0.5, 0.5, 0.5, 0.5, 0.5]</td>
        <td>[0.83, 1.00, 0.65, 0.64, 0.00]</td>
        <td>55%</td>
        <td>17%</td>
        <td>28%</td>
        <td>0,76</td>
    </tr>
    <tr style="background-color: #95dcd4;">
        <td>100</td>
        <td>100</td>
        <td>0.1</td>
        <td>[0.9, 0.9, 0.9, 0.9, 0.9]</td>
        <td>[0.00, 0.56, 1.00, 0.36, 0.29]</td>
        <td>48%</td>
        <td>34%</td>
        <td>18%</td>
        <td>0,58</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.1</td>
        <td>[0.1, 0.9, 0.1, 0.1, 0.1]</td>
        <td>[0.20, 0.68, 1.00, 0.32, 0.00]</td>
        <td>44%</td>
        <td>32%</td>
        <td>24%</td>
        <td>0,57</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.1</td>
        <td>[0.5, 0.5, 0.5, 0.5, 0.5]</td>
        <td>[0.21, 0.74, 0.96, 1.00, 0.00]</td>
        <td>50%</td>
        <td>28%</td>
        <td>22%</td>
        <td>0,64</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>Enfriamiento</td>
        <td>[0.9, 0.9, 0.9, 0.9, 0.9]</td>
        <td>[0.39, 0.80, 1.00, 0.53, 0.00]</td>
        <td>50%</td>
        <td>28%</td>
        <td>22%</td>
        <td>0,64</td>
    </tr>
    <tr style="background-color: #afdb99;">
        <td>100</td>
        <td>100</td>
        <td>Enfriamiento</td>
        <td>[0.1, 0.9, 0.1, 0.1, 0.1]</td>
        <td>[0.00, 0.39, 1.00, 0.53, 0.18]</td>
        <td>55%</td>
        <td>17%</td>
        <td>28%</td>
        <td>0,76</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>Enfriamiento</td>
        <td>[0.5, 0.5, 0.5, 0.5, 0.5]</td>
        <td>[1.00, 0.86, 0.96, 0.36, .00]</td>
        <td>31%</td>
        <td>6%</td>
        <td>63%</td>
        <td>0,83</td>
    </tr>
    <caption>Tabla 1 - Resultados de entrenamiento de <b>Instancia 1</b> para cada configuración paramétrica</caption>
</table>

#### 3.3.3.1.2. Resultados a lo largo de las iteraciones para distintas configuraciones paramétricas
***
A

<div style="margin-top: 16px; margin-bottom: 16px;">
    <div style="display: inline-block; width: 24%; text-align: center;">
        <img src="img/1_05_ResultsAll05.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.3 - Entrenamiento con ratio 0.5, pesos [0.5, 0.5, 0.5, 0.5, 0.5]</label>
    </div>
    <div style="display: inline-block; width: 24%; text-align: center;">
        <img src="img/1_ResultsAll05.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.2 - Entrenamiento con ratio 0.9, pesos [0.5, 0.5, 0.5, 0.5, 0.5]</label>
    </div>
    <div style="display: inline-block; width: 24%; text-align: center;">
        <img src="img/1_ResultsAll09.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.1 - Entrenamiento con ratio 0.1, pesos [0.9, 0.9, 0.9, 0.9, 0.9]</label>
    </div>
    <div style="display: inline-block; width: 24%; text-align: center;">
        <img src="img/1_ResultsAll01.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.3 - Entrenamiento con ratio variable, pesos [0.1, 0.9, 0.1, 0.1, 0.1]</label>
    </div>
</div>

**Observaciones:**
- A

#### 3.3.3.1.3. Error Cuadrático Medio cada 10 iteraciones para distintas configuraciones paramétricas
***

A

<div style="margin-top: 16px; margin-bottom: 24px;">
    <div style="display: inline-block; width: 24%; text-align: center;">
        <img src="img/1_05_ErrorsAll05.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.3 - Entrenamiento con ratio 0.5, pesos [0.5, 0.5, 0.5, 0.5, 0.5]</label>
    </div>
    <div style="display: inline-block; width: 24%; text-align: center;">
        <img src="img/1_ErrorAll05.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.2 - Entrenamiento con ratio 0.9, pesos [0.5, 0.5, 0.5, 0.5, 0.5]</label>
    </div>
    <div style="display: inline-block; width:24%; text-align: center;">
        <img src="img/1_ErrorsAll09.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.1 - Entrenamiento con ratio 0.1, pesos [0.9, 0.9, 0.9, 0.9, 0.9]</label>
    </div>
    <div style="display: inline-block; width: 24%; text-align: center;">
	<img src="img/1_ErrorAll01.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.3 - Entrenamiento con ratio variable, pesos [0.1, 0.9, 0.1, 0.1, 0.1]</label>
    </div>
</div>

**Observaciones:**
- A

### 3.4. Elección y comparación de representantes
***

#### 3.4.1. Métricas
***
Habiendo entrenado 24 jugadores distintos, resulta pertinente determinar de la forma menos errónea posible, cuales de ellos tuvieron un **mejor** rendimiento. Se utilizan dos métricas distintas con este fin:
- **Cantidad de Partidas Ganadas (_win rate_)**: Representa la cantidad de partidas ganadas sobre el total de partidas jugadas independientemente a su resultado, siguiendo la fórmula $$winRate = \frac{ganadas}{jugadas}$$
<br><br>
- **Ratio de Victorias (_victory rate_)**: Representa la cantidad de partidas ganadas sobre el total de partidas con algún resultado, es decir, no empatadas, siguiendo la fórmula $$victoryRate = \frac{ganadas}{ganadas + perdidas}$$
<br><br>
El motivo de tomar la segunda métrica además de la primera, es el hecho de que un jugador con más victorias que otro no es necesariamente mejor. Sea el escenario donde un **jugador 1** gana y pierde 50 partidas respectivamente y un **jugador 2** gana 25 y empata 75. Si bien **jugador 1** ganó el doble de partidas que **jugador 2**, perdió 50 partidas más. El motivo de los empates de **jugador 2** es indeterminado, por lo que no se puede decir quien de los dos es mejor, pero es importante tener en cuenta que **jugador 1 no es necesariamente mejor**.

#### 3.4.2. Elección y comparación
***
Una vez determinadas las métricas, se realizaron las siguientes gráficas para comparar utilizando la cantidad de partidas ganadas:

<div style="margin-top: 16px; margin-bottom: 16px;">
    <div style="display: inline-block; width: 49%; text-align: center;">
        <img src="img/instancia1.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.17 - Partidas ganadas para cada modelo entrenado de la instancia 1</label>
    </div>
    <div style="display: inline-block; width: 49%; text-align: center;">
        <img src="img/instancia2.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.18 - Partidas ganadas para cada modelo entrenado de la instancia 2</label>
    </div>
</div>

**Observaciones generales:**
- La **instancia 2** alcanzó rendimientos más altos que la **instancia 1**
- Observando la gráfica, los modelos de la **instancia 1** se encuentran en condiciones parejas, excepto por aquellos entrenados utilizando $\gamma = 0.5$.
- Observando la gráfica, los "mejores" modelos de la **instancia 2** son aquellos con un peor ratio de victorias en la tabla 2 (compensado por su cantidad de partidas ganadas). Se determina que el peso de la cantidad de partidas ganadas es mayor al del ratio de victorias en este caso, ya que los otros ejemplos presentan una gran cantidad de empates.

**Interpretaciones:**
- La **instancia 1** es más robusta en relación a las distintas configuraciones paramétricas. La **instancia 2** en cambio es más sensible, devolviendo mejores resultados solo para el vector inicial con pesos pequeños. Esto puede implicar que si se entrenase más tiempo ambas instancias, la **instancia 1** alcance un mejor rendimiento que la **instancia 2**.
- Descartando el punto anterior, se interpreta que la separación de atributos sin normalizar (**instancia 2**) tiene un mejor rendimiento. Esto puede deberse a la pérdida de información al agrupar $A_i, B_i, C_i, D_i$ en $A, B, C, D$, al hecho de haber utilizado **normalización mín-máx** en los pesos, lo cual genera que uno sea siempre 0 y otro siempre 1, o a alguna otra razón no considerada.
- Tomando el hecho de que el vector con mejores resultados en la **instancia 2** es $w = [0.1, -0.9, 0.9, -0.1, 0.1, 0.1, -0.1, -0.1, 0.1]$, se puede interpretar que priorizar $A_1$ y $A_2$ por sobre el resto de pesos es una práctica acertada para este problema.

## 4. Conclusiones
***

#### 4.1. Respecto a los Modelos
***
A

#### 4.2. Respecto a los Parámetros
***
A

#### 4.3. Respecto a los Resultados
***
A

#### 4.4. Posibles mejoras
***
A
- A